<a href="https://colab.research.google.com/github/annisapraditha/tugaspraktikum_ML3/blob/main/Kegiatan_2(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive #run
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ls #run

drive/  sample_data/


In [3]:
cd drive/MyDrive/DATASET/skin-lesions/ #run

/content/drive/MyDrive/DATASET/skin-lesions


In [4]:
ls #run

 modul3.h5                    'new split dataset 80:19:1'/   train/
'new split dataset 75:14:1'/   test/                         valid/


In [5]:
train_dir = "train" #run
test_dir = "test"
valid_dir = "valid"

In [6]:
!pip install split-folders

In [ ]:
import splitfolders  # or import split_folders #run

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("train", output="new split dataset 80:19:1", seed=1337, ratio=(.80, .19, .01), group_prefix=None) # default value

In [8]:
ls #run

 modul3.h5                    'new split dataset 80:19:1'/   train/
'new split dataset 75:14:1'/   test/                         valid/


In [9]:
ls "new split dataset 80:19:1" #run

test/  train/  val/


In [12]:
train_path = "new split dataset 80:19:1/train" #run
test_path  = "new split dataset 80:19:1/test"
val_path   = "new split dataset 80:19:1/val"

In [13]:
from keras.preprocessing.image import ImageDataGenerator #run

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
training_set = train_datagen.flow_from_directory(  train_path, #run
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = True)

Found 1599 images belonging to 3 classes.


In [15]:
validation_set = val_datagen.flow_from_directory(    val_path, #run
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 379 images belonging to 3 classes.


In [16]:
test_set = val_datagen.flow_from_directory(         test_path, #run
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 22 images belonging to 3 classes.


In [17]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENDEFINISIKAN MODEL DISINI
import tensorflow as tf #run
from tensorflow.keras import backend
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Activation, Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
# DEFINISIKAN MODEL ANDA DISINI #run
model = Sequential()

model.add(InputLayer(input_shape=[128,128,3]))

model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

In [19]:
# TULIS CODE UNTUK MENAMPILKAN SUMMARY DARI ARSITEKTUR MODEL ANDA DISINI
print(model.summary()) #run

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        73792     
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        9232      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 16)        0

In [20]:
# TULIS CODE UNTUK DEFINISI LEARNING RATE DISINI 
from tensorflow.keras.optimizers import Adam #run

model.compile(optimizer=Adam(lr=0.01), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# TULIS CODE UNTUK TRAINING DISINI
import time #run

start = time.time()
H = model.fit_generator(
        training_set,
        steps_per_epoch=44,
        epochs=100,
        validation_data=validation_set,
        validation_steps=5)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
44/44 [==============================] - 640s 14s/step - loss: 1.3303 - acc: 0.5913 - val_loss: 1.6211 - val_acc: 0.5562
Epoch 2/100
44/44 [==============================] - 306s 7s/step - loss: 0.8974 - acc: 0.6766 - val_loss: 0.9115 - val_acc: 0.5562
Epoch 3/100
44/44 [==============================] - 262s 6s/step - loss: 0.8333 - acc: 0.6944 - val_loss: 0.9375 - val_acc: 0.5562
Epoch 4/100
44/44 [==============================] - 245s 6s/step - loss: 0.8418 - acc: 0.6811 - val_loss: 0.9702 - val_acc: 0.5562
Epoch 5/100
44/44 [==============================] - 242s 6s/step - loss: 0.8353 - acc: 0.6802 - val_loss: 0.9524 - val_acc: 0.5562
Epoch 6/100
44/44 [==============================] - 227s 5s/step - loss: 0.8297 - acc: 0.6896 - val_loss: 0.9106 - val_acc: 0.5562
Epoch 7/100
44/44 [==============================] - 232s 5s/step - loss: 0.8444 - acc: 0.6766 - val_loss: 0.9408 - val_acc: 0.5562
Epoch 8/100
44/44 [==============================] - 228s 5s/step - loss: 0

In [ ]:
from keras.models import save_model #run
model = save_model(model,'modul3.h5')

In [ ]:
!ls

In [ ]:
# TULIS CODE UNTUK PLOT EVALUASI MODEL DISINI
plt.style.use("ggplot") #run
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:
pred2 = model2.predict(x_val) #run
labels2 = (pred2 > 0.5).astype(np.int)

print(classification_report(y_val, labels2))